#EN3150 Assignment 03: Simple convolutional neural network to perform classification.



##1 .CNN for image classification

### Mount the google Drive

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### Load the Dataset

In [4]:
# Import necessary libraries
import os
import cv2
import numpy as np
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt


# Define the base directory containing your images
base_dir = '/content/drive/My Drive/EN3150-Assignment-03-CNN/Images/'

# Lists to hold image data and labels
image_paths = []
labels = []

# Load images from each folder (H1, H2, H3, H5, H6)
for folder in os.listdir(base_dir):
    folder_path = os.path.join(base_dir, folder)
    if os.path.isdir(folder_path):
        for filename in os.listdir(folder_path):
            if filename.endswith('.jpg'):
                # Construct full image path
                image_path = os.path.join(folder_path, filename)
                image_paths.append(image_path)
                labels.append(folder)  # Assign label based on folder name

# Convert labels to a NumPy array
labels = np.array(labels)

### Split the Dataset

In [5]:
# Split data: 60% train, 20% validation, 20% test
train_paths, temp_paths, train_labels, temp_labels = train_test_split(
    image_paths, labels, test_size=0.4, random_state=42, stratify=labels
)
validation_paths, test_paths, validation_labels, test_labels = train_test_split(
    temp_paths, temp_labels, test_size=0.5, random_state=42, stratify=temp_labels
)

# Print the sizes of the splits
print(f'Training set: {len(train_paths)} images')
print(f'Validation set: {len(validation_paths)} images')
print(f'Test set: {len(test_paths)} images')

Training set: 5468 images
Validation set: 1823 images
Test set: 1823 images
